In [44]:
import pandas as pd

#Imports the cleaned questions
df = pd.read_csv('CleanedQuestions.csv')
df = df.loc[:, ~df.columns.str.contains('Unnamed')]

df

,Question,Name,Source
0,Test,Test,Test
1,"Hello,\n\nThe project is full of shit and is s...",uo3318,Reddit - r/Python
2,"This is more a ""Dammit! I hate my old eyes"" p...",unzzna,Reddit - r/Python
3,"Hello everyone, I am new to the coding realm a...",unzrcs,Reddit - r/Python
4,\n\n1. Code Everyday\n2. **Write It Out**: A...,unx8y0,Reddit - r/Python
...,...,...,...
1644,Small POS app using Python,fd7wxj,Reddit - r/Python
1645,[Google Brain Object detection] EfficientDet: ...,ek3ntu,Reddit - r/Python
1646,Django 2.0 Released,7h3bbh,Reddit - r/Python
1647,After trying several excel sheets and tools to...,p6e5va,Reddit - r/Python


In [45]:
#Asks the user to input their question and then see if they're question matches questions in the dataframe, it then extracts the name (or link) of the question's post that their query matched
#It then makes a second dataframe just containing the Name of that question and then converts the dataframe to a list and then sets a string variable to the first item in the list which is
#The question's name (or link)
question = input("Please enter your query")

df2 = (df['Name'].where(df['Question'] == question))

awnserList = df2.tolist()
awnserLink = awnserList[0]
print(awnserLink)